In [1]:
#!g1.1
%pip install librosa
%pip install wandb

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [2]:
#!g1.1
%pip install torch==1.10.0+cu111 torchaudio==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/torch_stable.html
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [12]:
#!g1.1
from typing import Tuple, Dict, Optional, List, Union
from itertools import islice

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import wandb
from typing import Tuple, Dict, Optional, List, Union
from itertools import islice

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from IPython import display
from dataclasses import dataclass

import torch
from torch import nn
import random
import torchaudio

import librosa
from matplotlib import pyplot as plt

In [108]:
#!g1.1
from generator import Generator
from utils.melspectrogram import MelSpectrogram, MelSpectrogramConfig
from utils.lj_dataset import Batch, LJSpeechCollator, LJSpeechDataset
from discriminator import Discriminator
from gan_training import GANTraining

In [37]:
#!g1.1
featurizer = MelSpectrogram(MelSpectrogramConfig())
dataset = LJSpeechDataset('.')
dataloader = DataLoader(LJSpeechDataset('.'), batch_size=3, collate_fn=LJSpeechCollator())

In [38]:
#!g1.1
batch = next(iter(dataloader))

In [40]:
#!g1.1
batch.waveform.shape, batch.waveforn_length, batch.melspec.shape

(torch.Size([3, 16384]),
 tensor([212893,  41885, 213149], dtype=torch.int32),
 torch.Size([3, 80, 65]))

In [99]:
#!g1.1
from generator_layers import *
class Generator(torch.nn.Module):
    def __init__(self, config):
        super(Generator, self).__init__()
        self.pre_net = weight_norm(nn.Conv1d(config.mel_h, config.n_out_initial_channels,
                                              kernel_size=7, stride=1, padding=3))
        n_upsamples = len(config.kernel_size)
        in_channels = [config.n_out_initial_channels // 2**i for i in range(n_upsamples)]
        self.net = nn.Sequential(*list(
            UpSampleBlock(in_channels[i], config.kernel_size[i], config.mrf_kernel_sizes, config.dilation, config.leaky)
            for i in range(n_upsamples)
        ))
        self.post_net = weight_norm(nn.Conv1d(in_channels=config.n_out_initial_channels // (2**len(config.kernel_size)),
                                               out_channels=1, kernel_size=7, stride=1, padding=3))
        self.init_weights()

    def init_weights(self):
        for name, module in self.named_modules():
            if type(module) is nn.Conv1d or type(module) is nn.Conv2d:
                with torch.no_grad():
                    module.weight.normal_(0, 0.01)

    def forward(self, x):
        x = self.pre_net(x)
        x = self.net(x)
        x = F.leaky_relu(x, 0.1)
        x = self.post_net(x).squeeze(1)
        x = F.tanh(x)
        return x


    def remove_weight_norm(self):
        print('Removing weight norm...')
        remove_weight_norm(self.pre_net)
        for l in self.net:
            l.remove_weight_norm()
        remove_weight_norm(self.post_net)

In [170]:
#!g1.1
loss_fn = nn.L1Loss()
featurizer_gpu = MelSpectrogram(MelSpectrogramConfig()).to(device)
def validate(model_gen, val_dataloader):
    model_gen.eval()
    losses = 0
    with torch.no_grad():
        for batch_num, batch in enumerate(val_dataloader):
            mel = batch.melspec.to(device)
            wav = batch.waveform.to(device)
            out_wav = model_gen(mel)
            out_mel = featurizer_gpu(out_wav)
            if out_mel.shape[-1] < mel.shape[-1]:
                out_mel = torch.cat([out_mel, -11.5129 * torch.ones((out_mel.shape[0], 80, mel.shape[-1] - out_mel.shape[-1])).to(device)], dim=-1)
            elif out_mel.shape[-1] > mel.shape[-1]:
                with torch.no_grad():
                    mel = torch.cat([mel, -11.5129 * torch.ones((mel.shape[0], 80, out_mel.shape[-1] - mel.shape[-1])).to(device)], dim=-1)
            loss = loss_fn(out_mel, mel)
            losses += loss.item()
            wandb.log({'train_mel_loss': loss.item()})
    with torch.no_grad():
        reconstructed_wav = out_wav.cpu()
        print(f"validated sampled_audios")
        display.display(display.Audio(reconstructed_wav[0], rate=22050))
        wandb.log({'val_reconstructed_wav0': wandb.Audio(reconstructed_wav[0].detach().cpu().numpy().T, sample_rate=22050),
                  'val_reconstructed_wav1': wandb.Audio(reconstructed_wav[1].detach().cpu().numpy().T, sample_rate=22050),
                  'val_reconstructed_wav2': wandb.Audio(reconstructed_wav[2].detach().cpu().numpy().T, sample_rate=22050),
                  'val_melspec0': wandb.Image(out_mel[0].detach().cpu().numpy()),
                  'val_melspec1': wandb.Image(out_mel[1].detach().cpu().numpy()),
                  'val_melspec2': wandb.Image(out_mel[2].detach().cpu().numpy())})
    
    return losses/len(val_dataloader)
        

In [109]:
#!g1.1
device = torch.device('cuda')
class ModelConfig:
    batch_size: int=3
    device = torch.device('cuda')
    num_workers: int = 4

    mel_h: int = 80
    n_out_initial_channels: int = 512

    leaky: float = 0.1

    kernel_size = [16, 16, 4, 4]
    mrf_kernel_sizes = [3, 7, 11]
    dilation = [[1,3,5], [1,3,5], [1,3,5]]


In [122]:
#!g1.1
dataset = LJSpeechDataset('.')
len(dataset)

13100

In [138]:
#!g1.1
test_size = 15
train_size = len(dataset) -  test_size#int(0.95 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [139]:
#!g1.1
train_dataloader = DataLoader(train_dataset, batch_size=3, collate_fn=LJSpeechCollator())
val_dataloader = DataLoader(test_dataset, batch_size=3, collate_fn=LJSpeechCollator())

In [140]:
#!g1.1
len(train_dataloader), len(val_dataloader)

(4362, 5)

In [172]:
#!g1.1
# wandb.init(name=RUN_NAME, project='hw4_hifi')
# log_train_audios(gan.model_gen)
val_losses = []
val_loss = validate(gan.model_gen, val_dataloader)
val_losses.append(val_loss)
print(f"Epoch val loss {val_loss}")

/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/functional.py:1795: UserWarning: nn.fu

validated sampled_audios
Epoch val loss 0.5518938183784485


In [116]:
#!g1.1
class DiscConfig:
    leaky: int = 0.1
    n_msd_layers: int = 3
    mpd_kernel_size: int = 5
    mpd_stride: int = 3
    periods: List[int] = [2, 3, 5, 7, 11]

In [161]:
#!g1.1
from gan_training5 import GANTraining
model_gen = Generator(ModelConfig()).to(device)
model_disc = Discriminator(DiscConfig()).to(device)
optimizer_gen = torch.optim.Adam(model_gen.parameters(), betas=(0.9, 0.98), lr=2e-4, eps=1e-9)
optimizer_disc = torch.optim.Adam(model_disc.parameters(), betas=(0.9, 0.98), lr=2e-4, eps=1e-9)
gan = GANTraining(model_gen, model_disc, optimizer_gen, optimizer_disc, scheduler_gen=None, scheduler_disc=None,
                 lambda_mel=45, lambda_wav=2, n_gen_steps=1)

In [141]:
#!g1.1
batch = next(iter(train_dataloader))

In [163]:
#!g1.1
def log_train_audios(model_gen):
    with torch.no_grad():
        mel = batch.melspec.to(device)
        wav = batch.waveform.to(device)
        out_wav = model_gen(mel).detach()
        out_mel = featurizer_gpu(out_wav).detach()
        reconstructed_wav = out_wav.cpu()
        print(f"step train sampled_audios")
        display.display(display.Audio(reconstructed_wav[0], rate=22050))
        wandb.log({'train_reconstructed_wav0': wandb.Audio(reconstructed_wav[0].detach().cpu().numpy().T, sample_rate=22050),
                  'train_reconstructed_wav1': wandb.Audio(reconstructed_wav[1].detach().cpu().numpy().T, sample_rate=22050),
                  'train_reconstructed_wav2': wandb.Audio(reconstructed_wav[2].detach().cpu().numpy().T, sample_rate=22050),
                  'train_melspec0': wandb.Image(out_mel[0].detach().cpu().numpy()),
                  'train_melspec1': wandb.Image(out_mel[1].detach().cpu().numpy()),
                  'train_melspec2': wandb.Image(out_mel[2].detach().cpu().numpy())})

In [ ]:
#!g1.1
import warnings
warnings.filterwarnings("ignore")
RUN_NAME = 'GAN_training_default'
wandb.init(name=RUN_NAME, project='hw4_hifi')
N_EPOCHS = 1000


train_gen_losses, train_disc_losses = [], []
val_losses = []
for epoch in range(N_EPOCHS):
    gen_loss, disc_loss = gan.train_step(train_dataloader)
    print(f"Epoch disc loss {disc_loss}, gen loss {gen_loss}")
    
    log_train_audios(gan.model_gen)
    val_loss = validate(gan.model_gen, val_dataloader)
    val_losses.append(val_loss)
    print(f"Epoch val loss {val_loss}")
    
    
    torch.save(gan.model_gen.state_dict(), RUN_NAME + '_gen_last_epoch.pth')
    torch.save(gan.model_disc.state_dict(), RUN_NAME + '_disc_last_epoch.pth')

    if epoch % 40 == 0 and epoch > 20 :
        torch.save(gan.model_gen.state_dict(), RUN_NAME + '_gen_last_epoch' + str(epoch) + '.pth')
        torch.save(gan.model_disc.state_dict(), RUN_NAME + '_disc_last_epoch' + str(epoch) + '.pth')

Epoch disc loss 3.1734327223256114, gen loss 31.935583058654142
step train sampled_audios
validated sampled_audios
Epoch val loss 0.5450081586837768
Epoch disc loss 3.088658367423277, gen loss 31.69046423053698
step train sampled_audios
validated sampled_audios
Epoch val loss 0.4683520793914795
Epoch disc loss 2.881873560246936, gen loss 32.342424054257314
step train sampled_audios
validated sampled_audios
Epoch val loss 0.5012366056442261
Epoch disc loss 2.666795978683881, gen loss 32.863960176474016
step train sampled_audios
validated sampled_audios
Epoch val loss 0.4440550088882446
Epoch disc loss 2.5532720915115736, gen loss 32.35474467681997
step train sampled_audios
validated sampled_audios
Epoch val loss 0.43253151774406434
Epoch disc loss 2.5185250475667913, gen loss 31.808343920123953
step train sampled_audios
validated sampled_audios
Epoch val loss 0.4253496527671814


Epoch disc loss 2.4455431715330365, gen loss 31.72556457012304
step train sampled_audios
validated sampled_audios
Epoch val loss 0.40995997190475464


Epoch disc loss 2.37530099905614, gen loss 31.65619501221459
step train sampled_audios
validated sampled_audios
Epoch val loss 0.3880841135978699


Epoch disc loss 2.3689706499070433, gen loss 31.342161865531715
step train sampled_audios
validated sampled_audios
Epoch val loss 0.4114354372024536


Epoch disc loss 2.3717507155922126, gen loss 31.147893160540594
step train sampled_audios
validated sampled_audios
Epoch val loss 0.3822993516921997


Epoch disc loss 2.349977390200979, gen loss 31.141759989623463
step train sampled_audios
validated sampled_audios
Epoch val loss 0.39665680527687075


Epoch disc loss 2.3787940790050897, gen loss 31.004073308291428
step train sampled_audios
validated sampled_audios
Epoch val loss 0.36920023560523985
Epoch disc loss 2.3843048153695356, gen loss 31.03036748920215
step train sampled_audios
validated sampled_audios
Epoch val loss 0.379035884141922
Epoch disc loss 2.3750737114873077, gen loss 30.937696301242944
step train sampled_audios
validated sampled_audios
Epoch val loss 0.3804316818714142
Epoch disc loss 2.3685661930657482, gen loss 30.94763467728354
step train sampled_audios
validated sampled_audios
Epoch val loss 0.36652728319168093
Epoch disc loss 2.3770779679533214, gen loss 30.890603291993838
step train sampled_audios
validated sampled_audios
Epoch val loss 0.35226086974143983
Epoch disc loss 2.3841997818190372, gen loss 30.966414577531356
step train sampled_audios
validated sampled_audios
Epoch val loss 0.36802504062652586
Epoch disc loss 2.3833918233685187, gen loss 30.999775391552003
step train sampled_audios
validated sampl

In [ ]:
#!g1.1


In [ ]:
#!g1.1


# Batch Overfit


In [ ]:
#!g1.1
device = torch.device('cuda')
class ModelConfig:
    batch_size: int=3
    device = torch.device('cuda')
    num_workers: int = 4

    mel_h: int = 80
    n_out_initial_channels: int = 512

    leakDiscriminatorat = 0.1

    kernel_size = [16, 16, 4, 4]
    mrf_kernel_sizes = [3, 7, 11]
    dilation = [[1,3,5], [1,3,5], [1,3,5]]

In [101]:
#!g1.1
model = Generator(ModelConfig())
model = model.to(device)

In [59]:
#!g1.1
out = model(batch.melspec.to(device))

/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [87]:
#!g1.1
inp.shape

torch.Size([3, 80, 65])

In [105]:
#!g1.1
inp = batch.melspec.to(device)
out_wav = model(inp)
out_wav.shape

torch.Size([3, 16640])

In [89]:
#!g1.1
batch.waveform.shape

torch.Size([3, 16384])

In [98]:
#!g1.1
wandb.init(name='Batch_overfit', project='hw4_hifi')
import warnings
warnings.filterwarnings("ignore")

reconstructed_wav = batch.waveform.cpu()
out_mel = batch.melspec
display.display(display.Audio(reconstructed_wav[0], rate=22050))
wandb.log({'batch_wav0': wandb.Audio(reconstructed_wav[0].detach().cpu().numpy(), sample_rate=22050),
          'batch_wav1': wandb.Audio(reconstructed_wav[1].detach().cpu().numpy(), sample_rate=22050),
          'batch_wav2': wandb.Audio(reconstructed_wav[2].detach().cpu().numpy(), sample_rate=22050),
          'batch_melspec0': wandb.Image(out_mel[0].detach().cpu().numpy()),
          'batch_melspec1': wandb.Image(out_mel[1].detach().cpu().numpy()),
          'batch_melspec2': wandb.Image(out_mel[2].detach().cpu().numpy())})

N_ITERATIONS = 10000
model.train()

loss_fn = nn.L1Loss()
# loss_fn_align = nn.L1Loss()
# loss_fn_align = nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), betas=(0.9, 0.98), lr=2e-4,
                                 eps=1e-9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.98999)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=N_ITERATIONS//10, epochs=10)

featurizer_gpu = MelSpectrogram(MelSpectrogramConfig()).to(device)
losses = 0
for i in range(N_ITERATIONS):
    inp = batch.melspec.to(device)
    gt = batch.waveform.to(device)

    out_wav = model(inp)
    
    out_mel = featurizer_gpu(out_wav)
    if out_mel.shape[-1] < inp.shape[-1]:
        out_mel = torch.cat([out_mel, -11.5129 * torch.ones((out_mel.shape[0], 80, inp.shape[-1] - out_mel.shape[-1])).to(device)], dim=-1)
    elif out_mel.shape[-1] > inp.shape[-1]:
        with torch.no_grad():
            inp = torch.cat([inp, -11.5129 * torch.ones((inp.shape[0], 80, out_mel.shape[-1] - inp.shape[-1])).to(device)], dim=-1)

    optimizer.zero_grad()
    
    loss = loss_fn(out_mel, inp)
    loss.backward()

    optimizer.step()
    scheduler.step()
    losses += loss.item()
    wandb.log({'train_mel_loss': loss.item()})
    if i % 500 == 0:
        with torch.no_grad():
            reconstructed_wav = out_wav.cpu()
            print(f"step {i} sampled_audios")
            display.display(display.Audio(reconstructed_wav[0], rate=22050))
            wandb.log({'reconstructed_wav0': wandb.Audio(reconstructed_wav[0].detach().cpu().numpy().T, sample_rate=22050),
                      'reconstructed_wav1': wandb.Audio(reconstructed_wav[1].detach().cpu().numpy().T, sample_rate=22050),
                      'reconstructed_wav2': wandb.Audio(reconstructed_wav[2].detach().cpu().numpy().T, sample_rate=22050),
                      'melspec0': wandb.Image(out_mel[0].detach().cpu().numpy()),
                      'melspec1': wandb.Image(out_mel[1].detach().cpu().numpy()),
                      'melspec2': wandb.Image(out_mel[2].detach().cpu().numpy())})

wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


step 0 sampled_audios
step 500 sampled_audios
step 1000 sampled_audios
step 1500 sampled_audios
step 2000 sampled_audios
step 2500 sampled_audios
step 3000 sampled_audios
step 3500 sampled_audios
step 4000 sampled_audios
step 4500 sampled_audios
step 5000 sampled_audios
step 5500 sampled_audios
step 6000 sampled_audios
step 6500 sampled_audios
step 7000 sampled_audios
step 7500 sampled_audios
step 8000 sampled_audios
step 8500 sampled_audios
step 9000 sampled_audios
step 9500 sampled_audios


In [ ]:
#!g1.1
